## Task 2 - Classification/Regression (Use all features)

Write a Python program to build a MLP classifier and evaluate your model’s performance. For this task, you should use the pre-processed and transformed dataset without feature selection from Task 3.1.

<span style="font-weight:bold;">Save your response (program and markdown) as task2.ipynb</span>

In [ ]:
# when to use linear regression

In [1]:
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt 
from functions.pre_processing import *
from functions.transformations import *
from functions.feature_selection import *
from functions.analysis import *
from functions.classifiers import *
from sklearn.metrics import accuracy_score

-----
## Pre-Processing and Transformation

In [2]:
x_train, x_test, y_train, y_test = fetch_datasets()
x_train_norm, x_test_norm = normalise_min_max(x_train, x_test)
# x_train_norm, x_test_norm = normalise_std_scaler(x_train, x_test)

# x_train_norm, x_test_norm = normalise_robust_scaler(x_train, x_test)



---
## MLP Classifier Training

In [3]:
mlp_training = mlp_classifier(x_train, x_test, y_train, y_test, "")

Testing Accuracy - [8. 9. 9. ... 3. 4. 2.]
Accuracy: 0.9255555555555556
Classification Report:
              precision    recall  f1-score   support

         0.0       0.93      0.94      0.93       613
         1.0       0.97      0.97      0.97       605
         2.0       0.91      0.93      0.92       628
         3.0       0.85      0.86      0.86       499
         4.0       0.96      0.94      0.95       507
         5.0       0.90      0.91      0.90       409
         6.0       0.85      0.87      0.86       481
         7.0       0.97      0.96      0.97       583
         8.0       0.92      0.87      0.89       511
         9.0       0.99      0.97      0.98       564

    accuracy                           0.93      5400
   macro avg       0.92      0.92      0.92      5400
weighted avg       0.93      0.93      0.93      5400



---
## Hyperparameter Optimisation

---
## Evaluation